In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# build data directory
data_dir = '../../data/stage1'
os.listdir(data_dir)

In [ ]:
# Load user profiles
churn_od_df = pd.read_csv(
    os.path.join(data_dir, 'Churn_OD.txt'), 
    delimiter = "|",
    encoding = "ISO-8859-1"
)

In [ ]:
churn_od_df.sort_values(by=['Demand_weight'], ascending = False).head(10)

In [ ]:
#churn_od_df.groupby(['County_of_Origin']).agg({'Demand_weight':'sum'})

## What is the most popular route from each county?

In [ ]:
df_max_demand_from = churn_od_df.groupby(['County_of_Origin']).agg({'Demand_weight':'max'})
df_max_demand_from = pd.merge(churn_od_df, df_max_demand_from, on=['County_of_Origin', 'Demand_weight'], how = 'inner')
df_max_demand_from[['County_of_Origin','County_of_Public_Transportation','Demand_weight']].sort_values(by=['Demand_weight'], ascending = False)

## Routes per County

In [ ]:
churn_od_df['County_of_Origin'] = churn_od_df['County_of_Origin'].str.upper()
(churn_od_df.groupby('County_of_Public_Transportation')
                           .agg({"County_of_Public_Transportation": 'count'})
                           .rename(columns={'County_of_Public_Transportation': 'number_of_routes'})
                           .sort_values(by=['number_of_routes'], ascending = False).head(10))
## most routes end at Lisboa followed by Porto
## numbers are high as there are multiple perish codes per county. Dont have perish codes for origin

In [ ]:
churn_od_df[churn_od_df['County_of_Public_Transportation'] == 'LISBOA']

#### Most routes end in Lisboa and Porto


## Internal routes

In [ ]:
churn_od_df['County_of_Origin'] = churn_od_df['County_of_Origin'].str.upper()
routes_count = (churn_od_df.groupby('County_of_Origin')
                           .agg({"County_of_Origin": 'count'})
                           .rename(columns={'County_of_Origin': 'number_of_routes'})
                           .sort_values(by=['number_of_routes'], ascending = False))

churn_od_df_internal = churn_od_df[churn_od_df['County_of_Public_Transportation'] == churn_od_df['County_of_Origin']]
churn_od_df_internal_agg = (churn_od_df_internal.groupby('County_of_Origin')
                    .agg({'County_of_Origin': 'count'})
                    .rename(columns={'County_of_Origin': 'number_of_routes_internal'}))
            

routes_count_final = pd.merge(routes_count, churn_od_df_internal_agg, left_index=True, right_index=True,  how = 'inner')
routes_count_final['percentage_of_routes_internal'] = routes_count_final['number_of_routes_internal']/ routes_count_final['number_of_routes']
routes_count_final['number_of_routes_external'] =  routes_count_final['number_of_routes'] - routes_count_final['number_of_routes_internal']
routes_count_final['percentage_of_routes_external'] =  1-  routes_count_final['percentage_of_routes_internal']
routes_count_final

In [ ]:
# can filter out return trips?

In [ ]:
pd.crosstab(churn_od_df.County_of_Origin , churn_od_df.County_of_Public_Transportation )